# Day 14: Parabolic Reflector Dish

## Part 1

You reach the place where all of the mirrors were pointing: a massive parabolic reflector dish attached to the side of another large mountain.

The dish is made up of many small mirrors, but while the mirrors themselves are roughly in the shape of a parabolic reflector dish, each individual mirror seems to be pointing in slightly the wrong direction. If the dish is meant to focus light, all it's doing right now is sending it in a vague direction.

This system must be what provides the energy for the lava! If you focus the reflector dish, maybe you can go where it's pointing and use the light to fix the lava production.

Upon closer inspection, the individual mirrors each appear to be connected via an elaborate system of ropes and pulleys to a large metal platform below the dish. The platform is covered in large rocks of various shapes. Depending on their position, the weight of the rocks deforms the platform, and the shape of the platform controls which ropes move and ultimately the focus of the dish.

In short: if you move the rocks, you can focus the dish. The platform even has a control panel on the side that lets you tilt it in one of four directions! The rounded rocks (`O`) will roll when the platform is tilted, while the cube-shaped rocks (`#`) will stay in place. You note the positions of all of the empty spaces (`.`) and rocks (your puzzle input). For example:

```
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
```

Start by tilting the lever so all of the rocks will slide north as far as they will go:

```
OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....
```

You notice that the support beams along the north side of the platform are damaged; to ensure the platform doesn't collapse, you should calculate the total load on the north support beams.

The amount of load caused by a single rounded rock (`O`) is equal to the number of rows from the rock to the south edge of the platform, including the row the rock is on. (Cube-shaped rocks (`#`) don't contribute to load.) So, the amount of load caused by each rock in each row is as follows:

```
OOOO.#.O.. 10
OO..#....#  9
OO..O##..O  8
O..#.OO...  7
........#.  6
..#....#.#  5
..O..#.O.O  4
..O.......  3
#....###..  2
#....#....  1
```

The total load is the sum of the load caused by all of the rounded rocks. In this example, the total load is `136`.

Tilt the platform so that the rounded rocks all roll north. Afterward, what is the total load on the north support beams?



In [22]:
import numpy as np


test_data_raw =(
"""O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""
)


def parse_data(data):
    return np.array([list(line) for line in data.splitlines()])


def print_data(data):
    for line in data:
        print(''.join(line))


test_data = parse_data(test_data_raw)
print_data(test_data)

O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....


In [23]:
def get_tilted_arrray(array, direction=0):
    SYMBOLS = ("O", ".")
    array_str = "".join(array)
    tilted_array_str = "#".join([
        SYMBOLS[direction] * subarr_str.count(SYMBOLS[direction]) + SYMBOLS[1 - direction] * subarr_str.count(SYMBOLS[1 - direction])
        for subarr_str in array_str.split("#")
    ])
    return list(tilted_array_str)


def tilt(data, axis, direction):
    arrays = []
    for idx in range(data.shape[axis]):
        array = np.take(data, idx, axis=axis)
        tilted_array = get_tilted_arrray(array, direction)
        arrays.append(tilted_array)
    stack = np.row_stack if axis == 0 else np.column_stack
    return stack(arrays)


def tilt_north(data):
    return tilt(data, 1, 0)


print_data(tilt_north(test_data))

OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....


In [24]:
def get_total_load(tilted_data):
    xs, _ = np.where(tilted_data == "O")
    return (tilted_data.shape[0] - xs).sum()


def part1(data):
    return get_total_load(tilt_north(data))


part1(test_data)

136

In [25]:
with open("input.txt") as f:
    data = parse_data(f.read())

print_data(data[:5, :5])

#....
O....
.....
.OO..
#....


In [26]:
part1(data)

108144

## Part 2

The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "spin cycle" attempts to do just that!

Each cycle tilts the platform four times so that the rounded rocks roll north, then west, then south, then east. After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.

Here's what happens in the example above after each of the first few cycles:

```
After 1 cycle:
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....

After 2 cycles:
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O

After 3 cycles:
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
```

This process should work if you leave it running long enough, but you're still worried about the north support beams. To make sure they'll survive for a while, you need to calculate the total load on the north support beams after `1000000000` cycles.

In the above example, after `1000000000` cycles, the total load on the north support beams is `64`.

Run the spin cycle for `1000000000` cycles. Afterward, what is the total load on the north support beams?

In [27]:
def cycle(data):
    for axis, direction in [(1, 0), (0, 0), (1, 1), (0, 1)]:
        data = tilt(data, axis, direction)
    return data


print_data(cycle(test_data))

.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....


In [28]:
def find_pattern(array):
    start = 0 if len(array) % 2 == 0 else 1
    for idx in range(start, len(array) - 2, 2):
        pattern_len = (len(array) - idx) // 2
        first = array[idx:idx + pattern_len]
        second = array[idx + pattern_len:idx + 2 * pattern_len]
        if np.all(first == second):
            return idx, pattern_len
    return 0, None


find_pattern(np.array([87, 69, 69, 69, 65, 64, 65, 63, 68, 69, 69, 65, 64, 65, 63, 68]))

(2, 7)

In [29]:
def part2(data, cycles=10**9):
    loads = []
    for _ in range(cycles):
        data = cycle(data)
        loads.append(get_total_load(data))
        prefix, pattern_len = find_pattern(loads)
        if pattern_len:
            load_idx = (cycles - prefix) % pattern_len
            return loads[prefix + load_idx - 1]
    return get_total_load(data)


part2(test_data)

64

In [30]:
part2(data)

108404